In [92]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

In [93]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg=Config.fromfile('./configs/yolof/yolof_r50_c5_8x8_1x_coco.py')
root='../../dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/retinanet_effb3_fpn_crop896_8x4_1x_coco'

#cfg.model.roi_head.bbox_head.num_classes = 10
cfg.model.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [94]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [95]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: ./work_dirs/retinanet_effb3_fpn_crop896_8x4_1x_coco/latest.pth


In [96]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 34.9 task/s, elapsed: 140s, ETA:     0s

In [97]:
print(output[0])

[array([[7.03426331e+02, 5.38276489e+02, 9.60573669e+02, 1.02400000e+03,
        2.24788681e-01],
       [6.16333374e+02, 4.92428894e+02, 9.18918213e+02, 8.89988953e+02,
        2.24118531e-01],
       [1.65300018e+02, 1.26177597e+02, 4.16230133e+02, 4.19396301e+02,
        2.23958805e-01],
       [2.69576294e+02, 7.66172409e+01, 4.29599854e+02, 2.98843964e+02,
        2.09116787e-01],
       [2.19711151e+02, 1.21756958e+02, 6.76288818e+02, 6.88307251e+02,
        1.95118815e-01],
       [2.12647736e+02, 1.28547684e+02, 5.10046478e+02, 4.97429626e+02,
        1.79435328e-01],
       [2.88295471e+02, 2.52454926e+02, 7.35704529e+02, 8.54069580e+02,
        1.75766617e-01],
       [1.52465515e+02, 5.21224548e+02, 3.33243683e+02, 7.34827087e+02,
        1.72886580e-01],
       [3.89102203e+02, 5.60603821e+02, 6.34897827e+02, 9.09038147e+02,
        1.72880396e-01],
       [1.97142624e+02, 3.50292664e+01, 4.49964844e+02, 3.31514038e+02,
        1.71505645e-01],
       [4.00854889e+02, 1.603

In [98]:
#output 시각화 (원래 없던 코드 내가 만든거)
#output[i][j] > i번째 이미지에 j번째 클래스 박스들 모음 [x1(왼쪽상단x좌표),y1(왼쪽상단),x2(오른쪽하단),y2,클래스에 속할 확률]
#현재는 전체 이미지에 박스를 씌운 이미지를 생성하는 코드
import cv2
class_names=["General trash","Paper","Paper pack","Metal","Glass","Plastic","Styrofoam","Plastic bag","Battery","Clothing"]
for i,image_info in enumerate(output):
    #image_id = image_info['id']
    #test dataset일때랑 train data일때는 'test'를 'train'으로 변경
    #숫자를 4자리 문자열로 바꿔줌 ( 3 >'0003')
    stridx = str(i).zfill(4)
    #test이미지가 있는 경로
    image_path = os.path.join('../../dataset/test',stridx+'.jpg')
    image = cv2.imread(image_path)
    #boxes에는 박스 좌표를, class_ids는 클래스 ids를 저장
    boxes=[]
    class_ids=[]
    for idx in range(10):
        for tempidx in range(len(output[i][idx])):
            boxes.append(output[i][idx][tempidx][:4])
            class_ids.append(idx)
           

    # 클래스 id와 색상 매핑
    class_colors = {
    0: (0, 255, 0),   # 클래스0에 대한 색상 (초록색)
    1: (0, 0, 255),   # 클래스1에 대한 색상 (빨간색)
    2: (255, 0, 0),   # 클래스2에 대한 색상 (파란색)
    3: (255, 255, 0), # 클래스3에 대한 색상 (노란색)
    4: (255, 0, 255), # 클래스4에 대한 색상 (자홍색)
    5: (0, 255, 255), # 클래스5에 대한 색상 (청록색)
    6: (128, 128, 0), # 클래스6에 대한 색상 (올리브색)
    7: (128, 0, 128), # 클래스7에 대한 색상 (보라색)
    8: (0, 128, 128), # 클래스8에 대한 색상 (청록색)
    9: (128, 128, 128)# 클래스9에 대한 색상 (회색)
}

    # 박스 시각화
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        class_id = class_ids[i]
        color = class_colors[class_id]
        class_name = class_names[class_id]
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        thickness = 2
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        cv2.putText(image, class_name, (int(x1), int(y1)-5), font, font_scale, color, thickness)

    # 결과 이미지 저장 (output 폴더안에 생성됨 inferenceoutput폴더 외에 다른 폴더에 만들고싶으면 ouput을 폴더명으로 바꺼주면댐)
    output_path = os.path.join('yolofinferenceoutput', 'infer_output_{}.jpg'.format(stridx))
    cv2.imwrite(output_path, image)

In [99]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.22478868 703.42633 538.2765 960.57367 1024...,test/0000.jpg
1,0 0.25482932 373.04507 282.0483 778.95496 634....,test/0001.jpg
2,0 0.18327518 24.15123 186.2038 168.37654 362.4...,test/0002.jpg
3,0 0.15579729 212.49176 726.0464 798.4705 937.9...,test/0003.jpg
4,0 0.23014045 361.17084 360.27197 677.7464 650....,test/0004.jpg


In [38]:
#train 데이터 확인해보기
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../../dataset/'

epoch = 'latest'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'train.json'
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=2021
cfg.gpu_ids = [1]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산
#output 시각화 (원래 없던 코드 내가 만든거)
#output[i][j] > i번째 이미지에 j번째 클래스 박스들 모음 [x1(왼쪽상단x좌표),y1(왼쪽상단),x2(오른쪽하단),y2,클래스에 속할 확률]
#현재는 전체 이미지에 박스를 씌운 이미지를 생성하는 코드
import cv2
class_names=["General trash","Paper","Paper pack","Metal","Glass","Plastic","Styrofoam","Plastic bag","Battery","Clothing"]
for i,image_info in enumerate(output):
    #image_id = image_info['id']
    #test dataset일때랑 train data일때는 'test'를 'train'으로 변경
    #숫자를 4자리 문자열로 바꿔줌 ( 3 >'0003')
    stridx = str(i).zfill(4)
    #test이미지가 있는 경로
    image_path = os.path.join('../../dataset/train',stridx+'.jpg')
    image = cv2.imread(image_path)

    boxes=[]
    class_ids=[]
    for idx in range(10):
        for tempidx in range(len(output[i][idx])):
            boxes.append(output[i][idx][tempidx][:4])
            class_ids.append(idx)
           

    # 클래스 id와 색상 매핑
    class_colors = {
    0: (0, 255, 0),   # 클래스0에 대한 색상 (초록색)
    1: (0, 0, 255),   # 클래스1에 대한 색상 (빨간색)
    2: (255, 0, 0),   # 클래스2에 대한 색상 (파란색)
    3: (255, 255, 0), # 클래스3에 대한 색상 (노란색)
    4: (255, 0, 255), # 클래스4에 대한 색상 (자홍색)
    5: (0, 255, 255), # 클래스5에 대한 색상 (청록색)
    6: (128, 128, 0), # 클래스6에 대한 색상 (올리브색)
    7: (128, 0, 128), # 클래스7에 대한 색상 (보라색)
    8: (0, 128, 128), # 클래스8에 대한 색상 (청록색)
    9: (128, 128, 128)# 클래스9에 대한 색상 (회색)
}

    # 박스 시각화
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = box
        class_id = class_ids[i]
        color = class_colors[class_id]
        class_name = class_names[class_id]
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1
        thickness = 2
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
        cv2.putText(image, class_name, (int(x1), int(y1)-5), font, font_scale, color, thickness)

    # 결과 이미지 저장 (output 폴더안에 생성됨 output폴더 외에 다른 폴더에 만들고싶으면 ouput을 폴더명으로 바꺼주면댐)
    output_path = os.path.join('trainoutput', 'train_output_{}.jpg'.format(stridx))
    cv2.imwrite(output_path, image)

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
load checkpoint from local path: ./work_dirs/faster_rcnn_r50_fpn_1x_trash/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4883/4883, 28.9 task/s, elapsed: 169s, ETA:     0s